In [ ]:
# STILL TESTING

from DataAccessCheck import DataChecker
from Scripts.SQLiteFun import create_sqlite_database, execution
from contextlib import closing #redirect_stdout
from IPython.utils.io import Tee

# once finished, try on all HAPI records and export output to file
# input abs path of database file you wish to query from
conn = create_sqlite_database("/home/jovyan/HDRL-Internship-2024/SPASE_Data_20240716.db")

# fails data checks 
HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 7"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 21"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 59"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 5 OFFSET 64"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 2 OFFSET 130"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 2 OFFSET 136"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 2 OFFSET 283""" #takes mult tries

# examples that are not actual datasets according to CDAWEB
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 3 OFFSET 120"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 14 OFFSET 233"""

# example w multiple prodKeys
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 45"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 67"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 93"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 96"""

# example where it passes data check at another interval besides 1sec
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 23"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 248""" #incorrect
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 4 OFFSET 258""" #incorrect

#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE '%/hapi' LIMIT 10"""

"""spase://NASA/NumericalData/AMPTE-CCE/MEPA/PT0.1875S
spase://NASA/NumericalData/DE2/IDM/PT0.25S
spase://NASA/NumericalData/FAST/MAG/Fluxgate/PT7.8125MS
spase://NASA/NumericalData/ISIS1/SFS/Ionogram/PT29S
spase://NASA/NumericalData/MESSENGER/MAG/PT0.05S"""

prodKeys = []
prodKeys = execution(HapiStmt, conn)
#print(prodKeys)

# if want output only in file
#with open("../DatalinkCheckOutput2.txt", "w") as file:
    #with redirect_stdout(file):
        #lines = DataChecker(prodKeys, conn)
# if want both file and console
with closing(Tee("../DatalinkCheckOutput2.txt", "w", channel="stdout")) as outputstream:
    lines = DataChecker(prodKeys, conn)
print("The program is done!")

# export all datasets which take too long to a text file for further investigation
textFile = open("../HAPI_TakeTooLong2.txt", "a")
for line in lines:
    textFile.write(line)
    textFile.write("\n")
textFile.close()

In [3]:
# call .py file directly from notebook
#%run ./HAPICheck.py > ../DatalinkCheckOutput.txt

In [4]:
# clear .txt file or use another file instead
#textFile = open("../HAPI_TakeTooLong2.txt", "w")
#textFile.close()

In [ ]:
# use this block to check results and add error msgs to records that should have them
from Scripts.SQLiteFun import executionALL

dataFails = 0
tookTooLong = 0
HAPIErrors = 0
with open("../DatalinkCheckOutput.txt") as file:
    f = file.readlines()
    for i, line in enumerate(f):
        if "No data was found" in line:
            dataFails += 1
            #print(i)
            info = f[i+1]
            before, sep, after = str(info).partition("[")
            rowNum, sep, after = after.partition(",")
            ErrorStmt = "HAPI info check passed after 1 attempt. HAPI data check"
            ErrorStmt += " failed after 9 attempts."""
            UpdateStmt = f""" UPDATE TestResults SET Errors = '{ErrorStmt}'
                               # WHERE rowNum = '{rowNum}' """
            executionALL(UpdateStmt, conn)
        if "Bad request - unknown dataset id" in line:
            HAPIErrors += 1
            info = f[i+2]
            before, sep, after = str(info).partition("[")
            rowNum, sep, after = after.partition(",")
            ErrorStmt = "HAPI info check failed"""
            UpdateStmt = f""" UPDATE TestResults SET Errors = '{ErrorStmt}'
                               # WHERE rowNum = '{rowNum}' """
            executionALL(UpdateStmt, conn)
with open("../HAPI_TakeTooLong.txt") as file:
    for line in file:
        tookTooLong += 1
print("The number of broken links is " + str(dataFails))
print("The number of links that are not actual datasets in CDAWeb are " + str(HAPIErrors))
tookTooLong -= (dataFails + HAPIErrors)
print("The number of links that timed out is " + str(tookTooLong))
dataSuccesses = 1632 - (tookTooLong + HAPIErrors + dataFails)
print("The number of links that successfully retrieved data are " + str(dataSuccesses))

In [ ]:
# use these to update Errors
key = "VOYAGER2_10S_MAG@0"
Stmt = f"SELECT SPASE_id FROM MetadataEntries WHERE prodKey LIKE '{key}%';"
ID = execution(Stmt, conn)
ID = ID[0]
print(ID)
Stmt = f"""UPDATE TestResults SET Errors = 'HAPI info check passed after 1 attempt. HAPI data check failed after 9 attempts.'
        WHERE SPASE_id = '{ID}' """;
execution(Stmt, conn)
Stmt = f"SELECT Errors FROM TestResults WHERE SPASE_id = '{ID}'; "
execution(Stmt, conn)

In [ ]:
# use these commands to check (missing 8 broken ones)
SELECT COUNT(Errors) FROM TestResults WHERE Errors LIKE "HAPI info check passed%";   
SELECT COUNT(Errors) FROM TestResults WHERE Errors LIKE "HAPI info check failed";   
SELECT COUNT(Errors) FROM TestResults WHERE Errors LIKE "HAPI info%";